<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Домашнее задание. Обучение языковой модели с помощью LSTM (10 баллов)

Э
В этом задании Вам предстоит обучить языковую модель с помощью рекуррентной нейронной сети. В отличие от семинарского занятия, Вам необходимо будет работать с отдельными словами, а не буквами.


Установим модуль ```datasets```, чтобы нам проще было работать с данными.

In [1]:
!pip install datasets

������� ������� ��������: 1251


Импорт необходимых библиотек

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List

import seaborn
seaborn.set(palette='summer')

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Воспользуемся датасетом imdb. В нем хранятся отзывы о фильмах с сайта imdb. Загрузим данные с помощью функции ```load_dataset```

In [5]:
# Загрузим датасет
dataset = load_dataset('imdb')

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

### Препроцессинг данных и создание словаря (1 балл)

Далее вам необходмо самостоятельно произвести препроцессинг данных и получить словарь или же просто ```set``` строк. Что необходимо сделать:

1. Разделить отдельные тренировочные примеры на отдельные предложения с помощью функции ```sent_tokenize``` из бибилиотеки ```nltk```. Каждое отдельное предложение будет одним тренировочным примером.
2. Оставить только те предложения, в которых меньше ```word_threshold``` слов.
3. Посчитать частоту вхождения каждого слова в оставшихся предложениях. Для деления предлоения на отдельные слова удобно использовать функцию ```word_tokenize```.
4. Создать объект ```vocab``` класса ```set```, положить в него служебные токены '\<unk\>', '\<bos\>', '\<eos\>', '\<pad\>' и vocab_size самых частовстречающихся слов.   

In [7]:
sentences = []
word_threshold = 32

# Получить отдельные предложения и поместить их в sentences
for sentence in tqdm(dataset['train']['text']):
    sentences.extend(
        [x.lower() for x in sent_tokenize(sentence, language='russian') if len(word_tokenize(x)) <= word_threshold]
    )

  0%|          | 0/25000 [00:00<?, ?it/s]

In [8]:
print("Всего предложений:", len(sentences))

Всего предложений: 204698


Посчитаем для каждого слова его встречаемость.

In [9]:
words = Counter()

for sentence in tqdm(sentences):
    words.update(word_tokenize(sentence))  # Используем word_tokenize для разбиения на слова

  0%|          | 0/204698 [00:00<?, ?it/s]

Добавим в словарь ```vocab_size``` самых встречающихся слов.

In [10]:
vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])
vocab_size = 40000

# Добавляем vocab_size самых частых слов (исключая уже добавленные служебные токены)
most_common_words = [word for word, count in words.most_common(vocab_size)]
vocab.update(most_common_words)

In [11]:
len(vocab)

40004

In [12]:
assert '<unk>' in vocab
assert '<bos>' in vocab
assert '<eos>' in vocab
assert '<pad>' in vocab
assert len(vocab) == vocab_size + 4

In [13]:
print("Всего слов в словаре:", len(vocab))

Всего слов в словаре: 40004


### Подготовка датасета (1 балл)

Далее, как и в семинарском занятии, подготовим датасеты и даталоадеры.

В классе ```WordDataset``` вам необходимо реализовать метод ```__getitem__```, который будет возвращать сэмпл данных по входному idx, то есть список целых чисел (индексов слов).

Внутри этого метода необходимо добавить служебные токены начала и конца последовательности, а также токенизировать соответствующее предложение с помощью ```word_tokenize``` и сопоставить ему индексы из ```word2ind```.

In [14]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [15]:
class WordDataset:
    def __init__(self, sentences):
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        sentence = self.data[idx]
        tokens = word_tokenize(sentence.lower())
        tokenized_sentence = [self.bos_id]
        # Допишите код здесь
        for token in tokens:
            tokenized_sentence.append(word2ind.get(token, self.unk_id))
        tokenized_sentence.append(self.eos_id)

        return tokenized_sentence

    def __len__(self) -> int:
        return len(self.data)

In [16]:
def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id=word2ind['<pad>']) -> torch.Tensor:
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:,:-1],
        'target_ids': sequences[:,1:]
    }

    return new_batch

In [17]:
train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2)
eval_sentences, test_sentences = train_test_split(sentences, test_size=0.5)

train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)
test_dataset = WordDataset(test_sentences)

batch_size = 128

train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

In [18]:
len(train_dataloader)

1280

## Обучение и архитектура модели

Вам необходимо на практике проверить, что влияет на качество языковых моделей. В этом задании нужно провести серию экспериментов с различными вариантами языковых моделей и сравнить различия в конечной перплексии на тестовом множестве.

Возмоэные идеи для экспериментов:

* Различные RNN-блоки, например, LSTM или GRU. Также можно добавить сразу несколько RNN блоков друг над другом с помощью аргумента num_layers. Вам поможет официальная документация [здесь](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
* Различные размеры скрытого состояния. Различное количество линейных слоев после RNN-блока. Различные функции активации.
* Добавление нормализаций в виде Dropout, BatchNorm или LayerNorm
* Различные аргументы для оптимизации, например, подбор оптимального learning rate или тип алгоритма оптимизации SGD, Adam, RMSProp и другие
* Любые другие идеи и подходы

После проведения экспериментов необходимо составить таблицу результатов, в которой описан каждый эксперимент и посчитана перплексия на тестовом множестве.

Учтите, что эксперименты, которые различаются, например, только размером скрытого состояния или количеством линейных слоев считаются, как один эксперимент.

Успехов!

### Функция evaluate (1 балл)

Заполните функцию ```evaluate```

In [19]:
def evaluate(model, criterion, dataloader) -> float:
    model.eval()
    perplexity = []
    with torch.no_grad():
        for batch in dataloader:
            logits = model(batch['input_ids']).flattem(start_dim=0, end_dim=1) # Посчитайте логиты предсказаний следующих слов
            loss = criterion(logits, batch['target_ids'].flatten())
            perplexity.append(torch.exp(loss).item())

    perplexity = sum(perplexity) / len(perplexity)

    return perplexity

### Train loop (1 балл)

Напишите функцию для обучения модели.

In [20]:
def train_model(model, train_loader, eval_loader, criterion, optimizer, device,
                epochs=10, patience=3, experiment_name='exp1'):
    """
    Функция обучения модели
    
    Параметры:
        model: Модель для обучения
        train_loader: DataLoader для обучающих данных
        eval_loader: DataLoader для валидационных данных
        criterion: Функция потерь
        optimizer: Оптимизатор
        device: Устройство (CPU/GPU)
        epochs: Максимальное количество эпох
        patience: Количество эпох для ранней остановки
        experiment_name: Название эксперимента
        
    Возвращает:
        Словарь с историей обучения
    """
    history = {
        'train_loss': [], 'eval_loss': [],
        'train_ppl': [], 'eval_ppl': []
    }
    best_eval_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        # Фаза обучения
        model.train()
        epoch_train_loss = 0
        #train_batches = 0--

        train_progress = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]')
        
        for batch in train_progress:
            if batch['input_ids'].size(1) == 0:
                continue
            optimizer.zero_grad()
            
            inputs = batch['input_ids'].to(device)
            targets = batch['target_ids'].to(device)
            
            logits = model(inputs)
            loss = criterion(logits.view(-1, logits.size(-1)), 
                          targets.view(-1))
            
            loss.backward()
            optimizer.step()
            
            epoch_train_loss += loss.item()
            train_progress.set_postfix({'loss': loss.item()})
        
        # Фаза валидации
        model.eval()
        eval_loss = 0
        with torch.no_grad():
            for batch in eval_loader:
                inputs = batch['input_ids'].to(device)
                targets = batch['target_ids'].to(device)
                
                logits = model(inputs)
                loss = criterion(logits.view(-1, logits.size(-1)), 
                              targets.view(-1))
                eval_loss += loss.item()
        
        # Вычисление метрик
        avg_train_loss = epoch_train_loss / len(train_loader)
        avg_eval_loss = eval_loss / len(eval_loader)
        train_ppl = torch.exp(torch.tensor(avg_train_loss)).item()
        eval_ppl = torch.exp(torch.tensor(avg_eval_loss)).item()
        
        # Сохранение истории
        history['train_loss'].append(avg_train_loss)
        history['eval_loss'].append(avg_eval_loss)
        history['train_ppl'].append(train_ppl)
        history['eval_ppl'].append(eval_ppl)
        
        print(f"{experiment_name} | Epoch {epoch+1}: "
              f"Train Loss: {avg_train_loss:.3f} (PPL: {train_ppl:.1f}) | "
              f"Eval Loss: {avg_eval_loss:.3f} (PPL: {eval_ppl:.1f})")
        
        # Ранняя остановка
        if avg_eval_loss < best_eval_loss:
            best_eval_loss = avg_eval_loss
            patience_counter = 0
            torch.save(model.state_dict(), f"best_{experiment_name}.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break
    
    return history

In [21]:
def train_model(model, train_loader, eval_loader, criterion, optimizer, device,
                epochs=10, patience=3, experiment_name='exp1'):
    history = {
        'train_loss': [], 'eval_loss': [],
        'train_ppl': [], 'eval_ppl': []
    }
    best_eval_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        epoch_train_loss = 0
        train_batches = 0
        
        train_progress = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]')
        
        for batch in train_progress:
            if batch['input_ids'].size(1) == 0:
                continue
                
            optimizer.zero_grad()
            
            inputs = batch['input_ids'].to(device)
            targets = batch['target_ids'].to(device)
            
            try:
                logits = model(inputs)
                # Исправлено: заменяем view на reshape
                loss = criterion(logits.reshape(-1, logits.size(-1)), 
                              targets.reshape(-1))
                
                loss.backward()
                optimizer.step()
                
                epoch_train_loss += loss.item()
                train_batches += 1
                train_progress.set_postfix({'loss': loss.item()})
            except RuntimeError as e:
                print(f"Skipping batch due to error: {str(e)}")
                continue
        
        if train_batches == 0:
            print("No valid batches in this epoch, skipping...")
            continue
            
        # Валидация
        model.eval()
        eval_loss = 0
        eval_batches = 0
        
        with torch.no_grad():
            for batch in eval_loader:
                if batch['input_ids'].size(1) == 0:
                    continue
                    
                inputs = batch['input_ids'].to(device)
                targets = batch['target_ids'].to(device)
                
                try:
                    logits = model(inputs)
                    # Исправлено: заменяем view на reshape
                    loss = criterion(logits.reshape(-1, logits.size(-1)), 
                                  targets.reshape(-1))
                    eval_loss += loss.item()
                    eval_batches += 1
                except RuntimeError as e:
                    print(f"Skipping eval batch due to error: {str(e)}")
                    continue
        
        if eval_batches == 0:
            print("No valid eval batches, skipping evaluation...")
            continue
            
        # Вычисление метрик
        avg_train_loss = epoch_train_loss / train_batches
        avg_eval_loss = eval_loss / eval_batches
        train_ppl = torch.exp(torch.tensor(avg_train_loss)).item()
        eval_ppl = torch.exp(torch.tensor(avg_eval_loss)).item()
        
        history['train_loss'].append(avg_train_loss)
        history['eval_loss'].append(avg_eval_loss)
        history['train_ppl'].append(train_ppl)
        history['eval_ppl'].append(eval_ppl)
        
        print(f"{experiment_name} | Epoch {epoch+1}: "
              f"Train Loss: {avg_train_loss:.3f} (PPL: {train_ppl:.1f}) | "
              f"Eval Loss: {avg_eval_loss:.3f} (PPL: {eval_ppl:.1f})")
        
        # Ранняя остановка
        if avg_eval_loss < best_eval_loss:
            best_eval_loss = avg_eval_loss
            patience_counter = 0
            torch.save(model.state_dict(), f"best_{experiment_name}.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break
    
    return history

In [22]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

class LanguageModel(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int = 300, 
                 hidden_dim: int = 512, num_layers: int = 2,
                 dropout: float = 0.3, rnn_type: str = 'lstm'):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=0
        )
        
        if rnn_type.lower() == 'lstm':
            self.rnn = nn.LSTM(
                embedding_dim,
                hidden_dim,
                num_layers=num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        elif rnn_type.lower() == 'gru':
            self.rnn = nn.GRU(
                embedding_dim,
                hidden_dim,
                num_layers=num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        else:
            raise ValueError(f"Unsupported RNN type: {rnn_type}")

        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)
        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.init_weights()

    def init_weights(self):
        init_range = 0.1
        self.embedding.weight.data.uniform_(-init_range, init_range)
        self.linear.weight.data.uniform_(-init_range, init_range)
        self.linear.bias.data.zero_()
        self.projection.weight.data.uniform_(-init_range, init_range)
        self.projection.bias.data.zero_()

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        # Проверка на пустые последовательности
        if input_batch.size(1) == 0:
            return torch.zeros(input_batch.size(0), 0, self.projection.out_features)
            
        embeddings = self.embedding(input_batch)
        embeddings = self.dropout(embeddings)
        output, _ = self.rnn(embeddings)
        output = self.layer_norm(output)
        output = self.non_lin(output)
        output = self.linear(output)
        output = self.dropout(output)
        logits = self.projection(output)
        return logits



In [23]:
def collate_fn_with_padding(batch):
    # Фильтрация пустых последовательностей
    batch = [x for x in batch if len(x) > 0]
    if not batch:
        return {'input_ids': torch.zeros(1, 1), 'target_ids': torch.zeros(1, 1)}
    
    # Находим максимальную длину в батче
    max_len = max(len(x) for x in batch)
    
    # Создаем тензоры с паддингом
    input_ids = []
    target_ids = []
    
    for sequence in batch:
        # Добавляем <bos> в начало и <eos> в конец
        padded_seq = [word2ind['<bos>']] + sequence + [word2ind['<eos>']]
        # Паддинг до максимальной длины
        padded_seq = padded_seq + [word2ind['<pad>']] * (max_len - len(sequence))
        
        input_ids.append(padded_seq[:-1])
        target_ids.append(padded_seq[1:])
    
    return {
        'input_ids': torch.LongTensor(input_ids),
        'target_ids': torch.LongTensor(target_ids)
    }

### Первый эксперимент (2 балла)

Определите архитектуру модели и обучите её.

In [24]:
if __name__ == "__main__":
    # Параметры модели
    VOCAB_SIZE = len(word2ind)  # Используем реальный размер словаря
    EMBEDDING_DIM = 300
    HIDDEN_DIM = 512
    NUM_LAYERS = 2
    DROPOUT = 0.3
    
    # Инициализация устройства
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Создание модели
    model = LanguageModel(
        vocab_size=VOCAB_SIZE,
        embedding_dim=EMBEDDING_DIM,
        hidden_dim=HIDDEN_DIM,
        num_layers=NUM_LAYERS,
        dropout=DROPOUT,
        rnn_type='lstm'
    ).to(device)
    
    # Оптимизатор и функция потерь
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
    
    # Обучение модели
    results = train_model(
        model=model,
        train_loader=train_dataloader,
        eval_loader=eval_dataloader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        epochs=15,
        patience=3,
        experiment_name='word_lm_experiment'
    )

Epoch 1/15 [Train]: 100%|██████████| 1280/1280 [20:29<00:00,  1.04it/s, loss=4.86]


word_lm_experiment | Epoch 1: Train Loss: 5.182 (PPL: 178.1) | Eval Loss: 4.624 (PPL: 101.9)


Epoch 2/15 [Train]: 100%|██████████| 1280/1280 [20:12<00:00,  1.06it/s, loss=4.51]


word_lm_experiment | Epoch 2: Train Loss: 4.634 (PPL: 102.9) | Eval Loss: 4.330 (PPL: 75.9)


Epoch 3/15 [Train]: 100%|██████████| 1280/1280 [20:45<00:00,  1.03it/s, loss=4.19]


...

word_lm_experiment | Epoch 13: Train Loss: 3.613 (PPL: 37.1) | Eval Loss: 3.473 (PPL: 32.2)


Epoch 14/15 [Train]: 100%|██████████| 1280/1280 [20:02<00:00,  1.06it/s, loss=2.87]


word_lm_experiment | Epoch 14: Train Loss: 3.578 (PPL: 35.8) | Eval Loss: 3.443 (PPL: 31.3)


Epoch 15/15 [Train]: 100%|██████████| 1280/1280 [20:02<00:00,  1.06it/s, loss=2.78]


word_lm_experiment | Epoch 15: Train Loss: 3.545 (PPL: 34.6) | Eval Loss: 3.411 (PPL: 30.3)


### Второй эксперимент (2 балла)

Попробуйте что-то поменять в модели или в пайплайне обучения, идеи для экспериментов можно подсмотреть выше.

In [27]:
if __name__ == "__main__":
    # Параметры модели
    VOCAB_SIZE = len(word2ind)  # Используем реальный размер словаря
    EMBEDDING_DIM = 200
    HIDDEN_DIM = 256
    NUM_LAYERS = 3
    DROPOUT = 0.2
    
    # Инициализация устройства
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Создание модели
    model = LanguageModel(
        vocab_size=VOCAB_SIZE,
        embedding_dim=EMBEDDING_DIM,
        hidden_dim=HIDDEN_DIM,
        num_layers=NUM_LAYERS,
        dropout=DROPOUT,
        rnn_type='gru'
    ).to(device)
    
    # Оптимизатор и функция потерь
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
    
    # Обучение модели
    results = train_model(
        model=model,
        train_loader=train_dataloader,
        eval_loader=eval_dataloader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        epochs=15,
        patience=3,
        experiment_name='word_lm_experiment'
    )

Epoch 1/15 [Train]: 100%|██████████| 1280/1280 [18:19<00:00,  1.16it/s, loss=5.38]


word_lm_experiment | Epoch 1: Train Loss: 5.904 (PPL: 366.3) | Eval Loss: 5.258 (PPL: 192.0)


Epoch 2/15 [Train]: 100%|██████████| 1280/1280 [18:45<00:00,  1.14it/s, loss=5.11]


word_lm_experiment | Epoch 2: Train Loss: 5.187 (PPL: 179.0) | Eval Loss: 4.964 (PPL: 143.2)


Epoch 3/15 [Train]: 100%|██████████| 1280/1280 [19:00<00:00,  1.12it/s, loss=4.98]


...

word_lm_experiment | Epoch 13: Train Loss: 4.472 (PPL: 87.6) | Eval Loss: 4.343 (PPL: 77.0)


Epoch 14/15 [Train]: 100%|██████████| 1280/1280 [19:38<00:00,  1.09it/s, loss=4.34]


word_lm_experiment | Epoch 14: Train Loss: 4.450 (PPL: 85.6) | Eval Loss: 4.317 (PPL: 74.9)


Epoch 15/15 [Train]: 100%|██████████| 1280/1280 [19:14<00:00,  1.11it/s, loss=4.3] 


word_lm_experiment | Epoch 15: Train Loss: 4.431 (PPL: 84.0) | Eval Loss: 4.307 (PPL: 74.2)


### Отчет (2 балла)

Опишите проведенные эксперименты. Сравните перплексии полученных моделей. Предложите идеи по улучшению качества моделей.

первая модель показала себя значительно лучше, но и обучалась немного дольше. для нахождения оптимальног решения необходимо ещё проводить эксперименты.